In [22]:
import os, json
import pandas as pd
import re
from pythainlp.util import thai_strptime
from datetime import datetime
import copy
import numpy as np

def iso8601_to_quarter(date_string):
    # แปลงจาก ISO8601 เป็น datetime object
    date_obj = datetime.fromisoformat(date_string)
    
    # คำนวณไตรมาส
    month = date_obj.month
    quarter = (month - 1) // 3 + 1  # ไตรมาส = (เดือน - 1) // 3 + 1
    
    # ส่งคืนผลลัพธ์
    return quarter
    # return f"Q{quarter} {date_obj.year}"

with open("data/provinces.json", 'r', encoding='utf-8') as file:
    data = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

for p in data:
    p["events"] = 0 
    p["events-q1"] = 0 
    p["events-q2"] = 0 
    p["events-q3"] = 0 
    p["events-q4"] = 0 

with open("data/analysis.json", 'r', encoding='utf-8') as file:
    analysis = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

data_detail = []
# SCORE EVENT
for a in analysis:
    ps = a["province"].split(" | ")
    for p in ps:
        for d in data:
            if d["province"] == p:
                new_d = copy.deepcopy(d)
                new_d["ISO8601"] = a["ISO8601"]
                new_d.pop("latitude", None)
                new_d.pop("longitude", None)
                new_d.pop("events", None)
                new_d["id"] = a["id"]
                new_d["title"] = a["title"]
                data_detail.append(new_d)
                d["events"] += 1
                quarter = iso8601_to_quarter(new_d["ISO8601"])
                d[f"events-q{quarter}"] += 1
        
# FILTER EVENTS ZERO
data = [x for x in data if x["events"] > 0]

# โหลดข้อมูลเป็น DataFrame
df = pd.DataFrame(data)
df["events-q12"] =df["events-q1"]+df["events-q2"]
df["events-q123"] =df["events-q1"]+df["events-q2"]+df["events-q3"]
df2 = pd.DataFrame(data_detail)

print(df)

import folium
from folium.plugins import HeatMap

min_intensity = min(df["events"])
max_intensity = max(df["events"])
# กำหนด Threshold Scale
threshold_scale = list(np.linspace(min_intensity, max_intensity, 5))

def export_heatmap(heat_data,filename):
    # สร้างแผนที่เริ่มต้น
    m = folium.Map(location=[13.7563, 100.5018], zoom_start=6)
    # เพิ่ม Heatmap ลงในแผนที่
    HeatMap(
        heat_data, 
        max_zoom=12,
        # min_opacity=0,  # โปร่งใสเมื่อความร้อนต่ำ
        # max_val=max_intensity,  # ใช้ค่าความร้อนสูงสุดเดียวกัน
        # threshold_scale=threshold_scale,
        ).add_to(m)
    # แสดงแผนที่
    m.save(filename)
    print(f"Heatmap saved as '{filename}'")

# เตรียมข้อมูลสำหรับ Heatmap 1
heat_data = [[row["latitude"], row["longitude"], row["events-q1"]] for index, row in df.iterrows()]
heat_data = [x for x in heat_data if x[2] > 0]
heat_data.append([-heat_data[0][0],-heat_data[0][1],max_intensity])
filename = "thailand_heatmap1.html"
export_heatmap(heat_data,filename)
# เตรียมข้อมูลสำหรับ Heatmap 2
heat_data = [[row["latitude"], row["longitude"], row["events-q12"]] for index, row in df.iterrows()]
heat_data = [x for x in heat_data if x[2] > 0]
heat_data.append([-heat_data[0][0],-heat_data[0][1],max_intensity])
filename = "thailand_heatmap2.html"
export_heatmap(heat_data,filename)
# เตรียมข้อมูลสำหรับ Heatmap 3
heat_data = [[row["latitude"], row["longitude"], row["events-q123"]] for index, row in df.iterrows()]
heat_data = [x for x in heat_data if x[2] > 0]
heat_data.append([-heat_data[0][0],-heat_data[0][1],max_intensity])
filename = "thailand_heatmap3.html"
export_heatmap(heat_data,filename)
# เตรียมข้อมูลสำหรับ Heatmap 4
heat_data = [[row["latitude"], row["longitude"], row["events"]] for index, row in df.iterrows()]
heat_data = [x for x in heat_data if x[2] > 0]
heat_data.append([-heat_data[0][0],-heat_data[0][1],max_intensity])
filename = "thailand_heatmap4.html"
export_heatmap(heat_data,filename)



filename = "data/heatmap.xlsx"
df.to_excel(filename, index=False)  # บันทึกเป็น Excel

filename = "data/analysis-detail.xlsx"
df2.to_excel(filename, index=False)  # บันทึกเป็น Excel

         province   latitude   longitude  events  events-q1  events-q2  \
0   กรุงเทพมหานคร  13.756539  100.555091       9          1          0   
1          กระบี่   8.014196   98.933196       3          0          0   
2       กาฬสินธุ์  16.589940  103.613395       2          0          0   
3         ขอนแก่น  16.354786  102.597114       4          0          0   
4        จันทบุรี  12.683440  102.114321       1          0          0   
..            ...        ...         ...     ...        ...        ...   
64        อ่างทอง  14.613110  100.382438      24          0          0   
65       อุดรธานี  17.407981  102.887755       6          0          0   
66      อุตรดิตถ์  17.639478  100.327030       3          0          0   
67      อุทัยธานี  15.371043   99.792609      12          0          0   
68    อุบลราชธานี  15.274664  105.023414       6          0          0   

    events-q3  events-q4  events-q12  events-q123  
0           6          2           1            7  
1      

In [20]:
heat_data

[[13.75653896, 100.5550909, 9],
 [8.014196429, 98.93319643, 3],
 [14.23315789, 99.43374737, 0],
 [16.5899403, 103.6133955, 2],
 [16.36767949, 99.6500641, 0],
 [16.35478607, 102.5971144, 4],
 [12.68344048, 102.1143214, 1],
 [13.68152688, 101.1431935, 0],
 [13.15496522, 101.0667304, 6],
 [15.15109434, 100.0795094, 37],
 [15.97548387, 101.9370726, 1],
 [10.33735345, 99.18531897, 5],
 [19.88275, 99.91875, 145],
 [18.85669347, 98.91724121, 102],
 [7.477913043, 99.5570087, 3],
 [12.11457407, 102.4941111, 4],
 [16.84886667, 98.83153333, 18],
 [14.18407143, 101.1309286, 1],
 [13.87528302, 100.1222358, 3],
 [17.31197895, 104.4824211, 15],
 [15.08368421, 102.1976842, 2],
 [8.347576471, 99.8606, 5],
 [15.67909231, 100.1862923, 16],
 [13.89219231, 100.4318462, 23],
 [6.231012987, 101.7508312, 2],
 [18.87981818, 100.8072323, 29],
 [18.15864151, 103.6755283, 15],
 [14.89128571, 102.9727672, 2],
 [14.04813333, 100.62965, 9],
 [11.88472917, 99.66945833, 2],
 [14.0006875, 101.5345, 6],
 [6.754756522, 1